In [1]:
import pandas as pd
import duckdb
import glob
import os

In [2]:
imd = pd.read_excel('data/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx', sheet_name='IMD2019')
imd.columns = imd.columns.str.strip().str.lower().str.replace(" ", "_")
imd = imd.rename(columns={"lsoa_code_(2011)": "LSOA11", 
                          "index_of_multiple_deprivation_(imd)_decile": "imd_decile"})[["LSOA11","imd_decile"]]
imd.to_parquet("data/processed/imd_2019.parquet", index=False)

In [5]:
file_path = r'C:\Users\khize\OneDrive\Documents\DataAnalytics\dataProjectsPortfolio\respiratoryAdmissions\data\raw\artificial_hes_apc_202302_v1_full\artificial_hes_apc_202302_v1_full'
con = duckdb.connect("data/pipeline.duckdb")

files = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith('.csv')]
files = files[-3:]

con.execute(f"""
COPY (
    SELECT 
        FYEAR,
        PARTYEAR,
        EPIKEY,
        PSEUDO_HESID,
        ADMIDATE,
        ADMIMETH,
        ADMISORC,
        CCG_GP_PRACTICE,
        DIAG_4_01,
        DIAG_4_02,
        DIAG_4_03,
        ETHNOS,
        FAE_EMERGENCY,
        RESCTY_ONS,
        SEX,
        DISDATE,
        SPELDUR,
        LSOA11
    FROM read_csv_auto({files}, SAMPLE_SIZE=-1)
) TO 'data/processed/data_3yrs.parquet' (FORMAT PARQUET);
""")

con.execute("""
COPY (
  SELECT A.*, B.imd_decile
  FROM 'data/processed/data_3yrs.parquet' A
  LEFT JOIN 'data/processed/imd_2019.parquet' B
  ON A.LSOA11 = B.LSOA11
) TO 'data/processed/data_3yrs_imd.parquet' (FORMAT PARQUET);
""")

In [4]:
df = pd.read_parquet(r"data\processed\data_3yrs_imd.parquet", engine="pyarrow")
df.head()

,FYEAR,PARTYEAR,EPIKEY,PSEUDO_HESID,ADMIDATE,ADMISORC,CCG_GP_PRACTICE,DIAG_4_01,DIAG_4_02,DIAG_4_03,ETHNOS,RESCTY_ONS,SEX,DISDATE,SPELDUR,LSOA11,imd_decile
0,1920,201999,565754447583,TESTo6pqmcTiyZmBONMehCfq3GyLxqVG,2019-04-12,19,07X,D128,I489,Z961,A,E99999999,1,2020-01-22,0.0,E01031514,6.0
1,1920,201999,227442184444,TESTo6pqmcTiyZmBONMehCfq3GyLxqVG,1801-01-01,19,07N,R69X,N736,Z991,A,E99999999,1,1801-01-01,0.0,E01019530,3.0
2,1920,201999,889838365690,TESTo6pqmcTiyZmBONMehCfq3GyLxqVG,2019-05-01,19,14Y,O365,Z491,E119,A,E99999999,1,2019-05-01,0.0,E01012411,5.0
3,1920,201999,93183021561,TESTo6pqmcTiyZmBONMehCfq3GyLxqVG,1801-01-01,19,00Y,I635,J351,K449,A,E99999999,1,1801-01-01,0.0,E01022670,4.0
4,1920,201999,499218726934,TESTo6pqmcTiyZmBONMehCfq3GyLxqVG,2019-05-19,19,05N,K317,Y544,F329,A,E99999999,1,2019-07-23,0.0,E01010670,3.0
